## Intro

Den här notebooken tar en xml-fil från Livsmedelsverket som innehåller en klassificering av alla livsmedel. 

Exemplet utgår ifrån https://docs.python.org/3/library/xml.etree.elementtree.html 

Syftet är att göra en array med livsmedelsnummer och "huvudgrupp" som sen kan läsas ner i databasen.

I slutet läses klassificeringen ner i databasen, och namnen på klasserna hamnar i en csv-fil, med antalet livsmedel i varje klass.

Ett livsmedel har den här strukturen i xml-filen:

```<Livsmedel>
    <Nummer>1082</Nummer>
    <Namn>Skinksås</Namn>
    <ViktGram>100</ViktGram>
    <Huvudgrupp>Sås dressing majonnäs </Huvudgrupp>
    <Klassificeringar>
        <Klassificering>
            <Namn>A Gruppindelning EuroFIR</Namn>
            <Varden>
                <KlassificeringVarde>
                    <Namn>Sås</Namn>
                    <Kod>A0862</Kod>
                </KlassificeringVarde>
            </Varden>
        </Klassificering>
        <Klassificering>
            <Namn>B Artklassificering</Namn>
            <Varden>
                <KlassificeringVarde>
                    <Namn>Ko</Namn>
                    <Kod>B1201</Kod>
                </KlassificeringVarde>
            </Varden>
        </Klassificering>
        <Klassificering>
            <Namn>C Del av växt/djur</Namn>
            <Varden>
                <KlassificeringVarde>
                    <Namn>Mjölk</Namn>
                    <Kod>C0235</Kod>
                </KlassificeringVarde>
            </Varden>
        </Klassificering>
        <Klassificering>
            <Namn>E Fysisk form</Namn>
            <Varden>
                <KlassificeringVarde>
                    <Namn>Vätska, hög viskositet, med fasta bitar</Namn>
                    <Kod>E0138</Kod>
                </KlassificeringVarde>
            </Varden>
        </Klassificering>
        <Klassificering>
            <Namn>F Värmebehandling</Namn>
            <Varden>
                <KlassificeringVarde>
                    <Namn>Fullständigt upphettad </Namn>
                    <Kod>F0014</Kod>
                </KlassificeringVarde>
            </Varden>
        </Klassificering>
        <Klassificering>
            <Namn>G Tillagning</Namn>
            <Varden>
                <KlassificeringVarde>
                    <Namn>Kokt</Namn>
                    <Kod>G0014</Kod>
                </KlassificeringVarde>
            </Varden>
        </Klassificering>
        <Klassificering>
            <Namn>H Industriell process</Namn>
            <Varden>
                <KlassificeringVarde>
                    <Namn>Ingrediens tillförd</Namn>
                    <Kod>H0225</Kod>
                </KlassificeringVarde>
            </Varden>
        </Klassificering>
        <Klassificering>
            <Namn>J Konserveringsmetod</Namn>
            <Varden>
                <KlassificeringVarde>
                    <Namn>Ej konserverad</Namn>
                    <Kod>J0003</Kod>
                </KlassificeringVarde>
            </Varden>
        </Klassificering>
        <Klassificering>
            <Namn>K Förpackningsmedium</Namn>
            <Varden>
                <KlassificeringVarde>
                    <Namn>Utan förpackningsmedium</Namn>
                    <Kod>K0003</Kod>
                </KlassificeringVarde>
            </Varden>
        </Klassificering>
        <Klassificering>
            <Namn>M Förpackningstyp</Namn>
            <Varden>
                <KlassificeringVarde>
                    <Namn>Opaketerad produkt</Namn>
                    <Kod>M0003</Kod>
                </KlassificeringVarde>
            </Varden>
        </Klassificering>
        <Klassificering>
            <Namn>N Förpackningsmaterial</Namn>
            <Varden>
                <KlassificeringVarde>
                    <Namn>Oförpackad</Namn>
                    <Kod>N0003</Kod>
                </KlassificeringVarde>
            </Varden>
        </Klassificering>
        <Klassificering>
            <Namn>P Etikettinfo</Namn>
            <Varden>
                <KlassificeringVarde>
                    <Namn>Människoföda</Namn>
                    <Kod>P0026</Kod>
                </KlassificeringVarde>
            </Varden>
        </Klassificering>
        <Klassificering>
            <Namn>R Land/område</Namn>
            <Varden>
                <KlassificeringVarde>
                    <Namn>Sverige</Namn>
                    <Kod>R0376</Kod>
                </KlassificeringVarde>
            </Varden>
        </Klassificering>
    </Klassificeringar>
</Livsmedel>```

Filen börjar med lite extra innan första livsmedlet kommer, så därför blir xml-trädets rot ett steg "ner".

```<?xml version="1.0" encoding="UTF-8"?>
    <LivsmedelDataset xmlns:xsd="http://www.w3.org/2001/XMLSchema" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance">
	<Version>2017-03-14T15:37:15.533</Version>
	<LivsmedelsLista>
		<Livsmedel>```

In [1]:
import numpy as np
import sqlite3 
import xml.etree.ElementTree as ET
tree = ET.parse('metadata_20171106.xml')
root = tree.getroot()[1]
# Hade det stått getroot()[0] hade man i stället fångat trädet som är Version...

In [2]:
root.tag

'LivsmedelsLista'

In [3]:
#För att testa att vi fått tag på rätt nivå kan vi skriva ut första "barnet" i root.
for child in root[0]:
    print(child.tag,child.text)
#Anledningen till att Klassificeringar blir None är att det är en stor struktur som 
#inte låter sig skrivas som "text"

Nummer 1
Namn Talg nöt
ViktGram 100
Huvudgrupp Övrigt fett (ister, talg, kokosfett)
Klassificeringar None


In [4]:
root[0][0].text #Man borde kunna gå igenom root snyggare än nedan...

'1'

In [5]:
b = np.array(root) 

i = 0
for Livsmedel in root.findall('Livsmedel'):
    number = Livsmedel.find('Nummer').text
    group = Livsmedel.find('Huvudgrupp').text
    b[i][0] = number
    b[i][1] = group
    i += 1

b = b[:,:2] #Kapa allt utom de två första kolumnerna
print(b)

[['1' 'Övrigt fett (ister, talg, kokosfett)']
 ['2' 'Övrigt fett (ister, talg, kokosfett)']
 ['3' 'Övrigt fett (ister, talg, kokosfett)']
 ..., 
 ['5973' 'Kryddor ']
 ['5974' 'Mjukt bröd ']
 ['6053' 'Nötter frön']]


## Läs ner i databasen

In [6]:
conn = sqlite3.connect('../livs.db')  # Create db and establish connection
conn.row_factory = sqlite3.Row
curs = conn.cursor()

columnObject = []

#Kolla om Huvudgrupp-kolumnen redan finns, annars lägg till den        
for row in conn.execute('PRAGMA table_info (livs);'):
    columnObject.append(row)
columns = np.array(columnObject)
newColumn = 'Huvudgrupp'
addColumn = True
for row in columns:
    print (row)
    if newColumn == row[1]:
        addColumn = False
        print (addColumn, newColumn)
if addColumn:
    #Man får se upp extra noga med kolumntypen, för den går inte att ändra i efterhand i sqlite...
    conn.execute('ALTER TABLE livs ADD COLUMN Huvudgrupp TEXT;')
conn.commit()

conn.close()

[0 'Livsmedelsnamn' 'TEXT' 0 None 0]
[1 'Livsmedelsnummer' 'TEXT' 0 None 0]
[2 'Energi_kcal' 'REAL' 0 None 0]
[3 'Energi_kJ' 'REAL' 0 None 0]
[4 'Kolhydrater_g' 'REAL' 0 None 0]
[5 'Fett_g' 'REAL' 0 None 0]
[6 'Protein_g' 'REAL' 0 None 0]
[7 'Fibrer_g' 'REAL' 0 None 0]
[8 'Vatten_g' 'REAL' 0 None 0]
[9 'Alkohol_g' 'REAL' 0 None 0]
[10 'Aska_g' 'REAL' 0 None 0]
[11 'Monosackarider_g' 'REAL' 0 None 0]
[12 'Disackarider_g' 'REAL' 0 None 0]
[13 'Sackaros_g' 'REAL' 0 None 0]
[14 'Fullkorn_totalt_g' 'REAL' 0 None 0]
[15 'Sockerarter_g' 'REAL' 0 None 0]
[16 'Summa_mattade_fettsyror_g' 'REAL' 0 None 0]
[17 'Fettsyra_4_0_10_0_g' 'REAL' 0 None 0]
[18 'Fettsyra_12_0_g' 'REAL' 0 None 0]
[19 'Fettsyra_14_0_g' 'REAL' 0 None 0]
[20 'Fettsyra_16_0_g' 'REAL' 0 None 0]
[21 'Fettsyra_18_0_g' 'REAL' 0 None 0]
[22 'Fettsyra_20_0_g' 'REAL' 0 None 0]
[23 'Summa_enkelomattade_fettsyror_g' 'REAL' 0 None 0]
[24 'Fettsyra_16_1_g' 'REAL' 0 None 0]
[25 'Fettsyra_18_1_g' 'REAL' 0 None 0]
[26 'Summa_fleromattade_fet

In [7]:
conn = sqlite3.connect('../livs.db')  # Create db and establish connection
conn.row_factory = sqlite3.Row
curs = conn.cursor()


for row in b:
    #print(row[1], row[0])
    #.rstrip() på nästa rad tar bort blanksteg i slutet på huvudgruppernas namn  
    sql = 'UPDATE livs SET Huvudgrupp = "'+row[1].rstrip()+'" WHERE Livsmedelsnummer = "'+row[0]+'";'
    print(sql)
    curs.execute(sql)

conn.commit()

conn.close()

UPDATE livs SET Huvudgrupp = "Övrigt fett (ister, talg, kokosfett)" WHERE Livsmedelsnummer = "1";
UPDATE livs SET Huvudgrupp = "Övrigt fett (ister, talg, kokosfett)" WHERE Livsmedelsnummer = "2";
UPDATE livs SET Huvudgrupp = "Övrigt fett (ister, talg, kokosfett)" WHERE Livsmedelsnummer = "3";
UPDATE livs SET Huvudgrupp = "Övrigt fett (ister, talg, kokosfett)" WHERE Livsmedelsnummer = "4";
UPDATE livs SET Huvudgrupp = "Hård matfettsblandning" WHERE Livsmedelsnummer = "5";
UPDATE livs SET Huvudgrupp = "Hård matfettsblandning" WHERE Livsmedelsnummer = "6";
UPDATE livs SET Huvudgrupp = "Hård matfettsblandning" WHERE Livsmedelsnummer = "7";
UPDATE livs SET Huvudgrupp = "Hård matfettsblandning" WHERE Livsmedelsnummer = "8";
UPDATE livs SET Huvudgrupp = "Hård matfettsblandning" WHERE Livsmedelsnummer = "9";
UPDATE livs SET Huvudgrupp = "Flytande matfettsblandning" WHERE Livsmedelsnummer = "10";
UPDATE livs SET Huvudgrupp = "Hård matfettsblandning" WHERE Livsmedelsnummer = "12";
UPDATE livs SE

UPDATE livs SET Huvudgrupp = "Grönsaks- rotfrukts- baljväxträtter och produkter" WHERE Livsmedelsnummer = "306";
UPDATE livs SET Huvudgrupp = "Soppa mat" WHERE Livsmedelsnummer = "307";
UPDATE livs SET Huvudgrupp = "Soppa mat" WHERE Livsmedelsnummer = "308";
UPDATE livs SET Huvudgrupp = "Rotfrukter" WHERE Livsmedelsnummer = "310";
UPDATE livs SET Huvudgrupp = "Grönsaks- rotfrukts- baljväxträtter och produkter" WHERE Livsmedelsnummer = "314";
UPDATE livs SET Huvudgrupp = "Grönsaks- rotfrukts- baljväxträtter och produkter" WHERE Livsmedelsnummer = "315";
UPDATE livs SET Huvudgrupp = "Soppa mat" WHERE Livsmedelsnummer = "317";
UPDATE livs SET Huvudgrupp = "Rotfrukter" WHERE Livsmedelsnummer = "319";
UPDATE livs SET Huvudgrupp = "Grönsaker" WHERE Livsmedelsnummer = "320";
UPDATE livs SET Huvudgrupp = "Grönsaker" WHERE Livsmedelsnummer = "321";
UPDATE livs SET Huvudgrupp = "Grönsaker" WHERE Livsmedelsnummer = "322";
UPDATE livs SET Huvudgrupp = "Grönsaker" WHERE Livsmedelsnummer = "323";
UP

UPDATE livs SET Huvudgrupp = "Gröt" WHERE Livsmedelsnummer = "654";
UPDATE livs SET Huvudgrupp = "Gröt" WHERE Livsmedelsnummer = "655";
UPDATE livs SET Huvudgrupp = "Fruktjuice mm" WHERE Livsmedelsnummer = "657";
UPDATE livs SET Huvudgrupp = "Fruktjuice mm" WHERE Livsmedelsnummer = "658";
UPDATE livs SET Huvudgrupp = "Fruktjuice mm" WHERE Livsmedelsnummer = "659";
UPDATE livs SET Huvudgrupp = "Fruktjuice mm" WHERE Livsmedelsnummer = "660";
UPDATE livs SET Huvudgrupp = "Fruktjuice mm" WHERE Livsmedelsnummer = "661";
UPDATE livs SET Huvudgrupp = "Fruktjuice mm" WHERE Livsmedelsnummer = "664";
UPDATE livs SET Huvudgrupp = "Fruktjuice mm" WHERE Livsmedelsnummer = "665";
UPDATE livs SET Huvudgrupp = "Fruktjuice mm" WHERE Livsmedelsnummer = "666";
UPDATE livs SET Huvudgrupp = "Fruktjuice mm" WHERE Livsmedelsnummer = "668";
UPDATE livs SET Huvudgrupp = "Fruktjuice mm" WHERE Livsmedelsnummer = "670";
UPDATE livs SET Huvudgrupp = "Fruktjuice mm" WHERE Livsmedelsnummer = "671";
UPDATE livs SET H

OperationalError: near "smaksättare": syntax error

In [8]:
c = b[:,1:]
unique, counts = np.unique(c, return_counts=True) #https://stackoverflow.com/questions/28663856/how-to-count-the-occurrence-of-certain-item-in-an-ndarray-in-python
huvudgrupper = dict(zip(unique, counts))
print(huvudgrupper)
len(unique)

{'Bär färska frysta': 24, 'Dryck': 1, 'Sallad blandad mat': 10, 'Dessertost': 8, 'Buljong': 18, 'Nötter frön': 22, 'Grädde creme fraiche': 23, 'Blodmat': 2, 'Fågelprodukter o rätter': 5, 'Mjölk': 11, 'Glass': 25, 'Grönsaks- rotfrukts- baljväxträtter och produkter': 55, 'Fisk färsk fryst kokt': 68, 'Potatisprodukter o rätter': 33, 'Ättika vinäger': 3, 'Äggprodukter o rätter': 6, 'Deg och gräddade skal och bottnar': 3, 'Kryddor ': 1, 'Mjukt bröd ': 42, 'Sojaprotein veteprotein Quorn produkter och rätter': 3, 'Rom, kaviar': 6, 'Te': 4, 'Mjölkdryck chokladdryck milkshake smothie m yoghurt': 5, 'Välling': 12, 'Pasta': 21, 'Ägg ': 9, 'Övrigt fett (ister, talg, kokosfett)': 4, 'Gelatin agar agar': 2, 'Tacoskal': 1, 'Hårt bröd ': 28, 'Skaldjur bläckfisk färsk fryst kokt': 14, 'Vin': 10, 'Kost- o näringspreparat': 8, 'Vegetabiliska produkter och mjölkersättning': 13, 'Köttprodukter kötträtter': 80, 'Inälvor och organ': 33, 'Söta soppor kräm o efterrättssås': 26, 'Ris risnudlar': 25, 'Osträtter'

118

In [12]:
import csv

Det är lättare att stoppa här, och lämna analysen av kategorierna till ett annat underprojekt.

Två saker att se upp med:
* Det är ändå några kategorier som har blanksteg efter sig, som "Ägg " och "Grönsaker ". De borde ha tagits bort av rstrip()
* De kategorinamn som har citationstecken i sig kommer nog att ställa till problem senare...

In [17]:
with open('dict.csv', 'w') as csv_file: #https://stackoverflow.com/questions/8685809/python-writing-a-dictionary-to-a-csv-file-with-one-line-for-every-key-value
    #Filen behövde öppnas med w och inte wb för det är ingen binärfil.
    writer = csv.writer(csv_file)
    for key, value in huvudgrupper.items():
        writer.writerow([key, value])
        #print([key, value])